In [1]:
import os

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import analysis.utils as u

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


In [2]:
# PATHS
LUND_PATH = os.path.join(u.OUTPUT_DIR, "article_results", "lund2013", "figures")
HFC_PATH = os.path.join(u.OUTPUT_DIR, "article_results", "hfc", "figures")
OUT_PATH = os.path.join(u.BASE_DIR, "article", "poster", "figures2")
os.makedirs(OUT_PATH, exist_ok=True)

# VISUALIZATION SETTINGS
GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 3 * GRID_LINE_WIDTH

SINGLE_MEASURE_OPACITY, MEDIAN_OPACITY = 0.2, 1.0
SINGLE_MEASURE_LINE = dict(width=2 * GRID_LINE_WIDTH)
SINGLE_MEASURE_MARKER = dict(symbol="circle-open", size=6 * GRID_LINE_WIDTH,)
MEADIAN_LINE = dict(width=SINGLE_MEASURE_LINE['width'])
MEADIAN_MARKER = dict(symbol="circle", size=2 * SINGLE_MEASURE_MARKER['size'],)

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=28, color=FONT_COLOR)
SUBTITLE_FONT = dict(family=FONT_FAMILY, size=24, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_LABEL_STANDOFF = 3

## pEYES Example - Single Trial

In [3]:
with open(os.path.join(LUND_PATH, "supp_fig_A1.json"), 'r') as f:
    peyes_example_fig = pio.from_json(f.read())

peyes_example_fig.write_image(os.path.join(OUT_PATH, "peyes_example_fig.png"))
peyes_example_fig.show()

## Sample-Level Agreement

In [4]:
with open(os.path.join(LUND_PATH, "fig_2-RA.json"), 'r') as f:
    lund_fig = pio.from_json(f.read())
with open(os.path.join(HFC_PATH, "fig1-sample_agreement.json"), 'r') as f:
    hfc_fig = pio.from_json(f.read())


sample_fig = make_subplots(
    rows=1, cols=2, shared_yaxes=True,
    subplot_titles=["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"],
    horizontal_spacing=0.025
)
for tr in lund_fig.data:
    if tr["scalegroup"] != "cohen's_kappa":
        continue
    tr["scalegroup"] = "lund"
    tr["meanline"] = {'color': 'lightgray', 'visible': True, 'width': 3}
    sample_fig.add_trace(
        tr,
        row=1, col=1
    )
for tr in hfc_fig.data:
    if tr["scalegroup"] != "cohen's_kappa":
        continue
    if tr["yaxis"] != "y2":
        # only use RA as GT
        continue
    if tr["name"] == "2<sup>nd</sup> Ann.":
        tr["name"] = tr["legendgroup"] = tr["y0"] = "Ann. MN"
    tr["scalegroup"] = "hfc"
    sample_fig.add_trace(
        tr,
        row=1, col=2
    )

sample_fig.for_each_trace(lambda tr: tr.update(width=1.75))
sample_fig.for_each_annotation(lambda ann: ann.update(font=SUBTITLE_FONT, y=1.0, yanchor="bottom"))
sample_fig.for_each_xaxis(lambda xax: xax.update(
    title=dict(text="Cohen's <i>K</i>", font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
    range=[-0.02, 1.01],
    showline=False,
    showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT,
))
sample_fig.update_yaxes(
    title=dict(text="Detector", font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF), row=1, col=1
)
sample_fig.for_each_yaxis(lambda yax: yax.update(
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT,
))
sample_fig.update_layout(
    width=1500, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, b=0, t=25, pad=0),
)

sample_fig.write_image(os.path.join(OUT_PATH, "sample_agreement.png"), scale=3,)
sample_fig.show()

## Fixation Sensitivity over $\Delta t$

In [5]:
with open(os.path.join(LUND_PATH, "fig4-RA.json"), 'r') as f:
    lund_fix_fig = pio.from_json(f.read())
with open(os.path.join(HFC_PATH, "fixation_boundary_sensitivity-RA.json"), 'r') as f:
    hfc_fix_fig = pio.from_json(f.read())

row_titles = ["Onset <i>d'</i>", "Offset <i>d'</i>"]
column_title = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
fix_sensitivity_fig = make_subplots(
    rows=len(row_titles), cols=len(column_title),
    row_titles=row_titles, column_titles=column_title,
    shared_xaxes=True, shared_yaxes=True,
    horizontal_spacing=0.01, vertical_spacing=0.02,
)

# add traces
for tr in lund_fix_fig.data:
    if tr["yaxis"] not in {"y", "y2"}:  # ignore bottom-row violin traces
        continue
    tr["error_y"]["thickness"] = 0.5
    fix_sensitivity_fig.add_trace(tr, col=1, row=1 if tr["yaxis"] == "y" else 2,)
for tr in hfc_fix_fig.data:
    if tr["yaxis"] not in {"y", "y2"}:  # ignore bottom-row violin traces
        continue
    tr["error_y"]["thickness"] = 0.5
    tr["showlegend"] = False  # hide legend
    fix_sensitivity_fig.add_trace(tr, col=2, row=1 if tr["yaxis"] == "y" else 2,)

# update layout
fix_sensitivity_fig.for_each_annotation(lambda ann: ann.update(
    font=SUBTITLE_FONT if ann.text in column_title else AXIS_LABEL_FONT,
    textangle=0 if ann.text in column_title else -90,
    y=1.0 if ann.text in column_title else ann['y'], yanchor="bottom" if ann.text in column_title else ann['yanchor'],
    x=ann['x'] if ann.text in column_title else -0.02, xanchor=ann["xanchor"] if ann.text in column_title else "right",
))
fix_sensitivity_fig.for_each_yaxis(lambda yax: yax.update(
    title=dict(font=AXIS_LABEL_FONT),
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    range=[-1.4, 5.5],
    tickfont=AXIS_TICK_FONT, tickmode="linear", tick0=0, dtick=1.5,
))
fix_sensitivity_fig.for_each_xaxis(lambda xax: xax.update(
    title=dict(
        text="Δt (<i>ms</i>)"  if xax["anchor"] in {'y3', 'y4'} else "",
        font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF
    ),
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT, tickmode="array", tickvals=np.arange(0, 21, 5),
    ticktext=[f"{(3.33*v):.1f}" for v in np.arange(0, 21, 5)] if xax["anchor"]=="y4" else [f"{(2*v):.1f}" for v in np.arange(0, 21, 5)],
))
fix_sensitivity_fig.update_layout(
    width=1500, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=52, r=0, b=0, t=25, pad=0),
    legend=dict(
        orientation="h", bgcolor='rgba(0, 0, 0, 0)',
        yanchor="top", y=-0.125, xanchor="center", x=0.5,
        font=AXIS_TICK_FONT, itemwidth=90,
    ),
)

# save and show figure
fix_sensitivity_fig.write_image(os.path.join(OUT_PATH, "fix_sensitivity.png"), scale=3,)
fix_sensitivity_fig.show()

## Saccade Sensitivity over $\Delta t$

In [6]:
with open(os.path.join(LUND_PATH, "fig5-RA.json"), 'r') as f:
    lund_sac_fig = pio.from_json(f.read())

row_titles = ["Onset <i>d'</i>", "Offset <i>d'</i>"]
column_titles = ["<b><i>lund2013</i></b>"]
sac_sensitivity_fig = make_subplots(
    rows=len(row_titles), cols=len(column_titles),
    row_titles=row_titles, column_titles=column_titles,
    shared_xaxes=True, shared_yaxes=False,
    horizontal_spacing=0.01, vertical_spacing=0.02,
)

# add traces
for tr in lund_sac_fig.data:
    if tr["yaxis"] not in {"y", "y2"}:  # ignore bottom-row violin traces
        continue
    tr["error_y"]["thickness"] = 0.5
    sac_sensitivity_fig.add_trace(tr, col=1, row=1 if tr["yaxis"] == "y" else 2,)

# update layout
sac_sensitivity_fig.for_each_annotation(lambda ann: ann.update(
    font=SUBTITLE_FONT,
    textangle=-90 if ann.text in row_titles else 0,
    x=-0.05 if ann.text in row_titles else ann['x'], xanchor="right" if ann.text in row_titles else ann['xanchor'],
    y=1.0 if ann.text in column_titles else ann['y'], yanchor="bottom" if ann.text in column_titles else ann['yanchor'],
))
sac_sensitivity_fig.for_each_yaxis(lambda yax: yax.update(
    title=dict(font=AXIS_LABEL_FONT),
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    range=[-1.4, 5.5],
    tickfont=AXIS_TICK_FONT, tickmode="linear", tick0=0, dtick=1.5,
))
sac_sensitivity_fig.for_each_xaxis(lambda xax: xax.update(
    title=dict(text="Δt (<i>ms</i>)" if xax["anchor"]=="y2" else "", font=AXIS_LABEL_FONT, standoff=2*AXIS_LABEL_STANDOFF),
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT, tickmode="array", tickvals=np.arange(0, 21, 5),
    ticktext=[f"{(2*v):.1f}" for v in np.arange(0, 21, 5)],
))
sac_sensitivity_fig.update_layout(
    width=750, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=57.5, r=0, b=0, t=30, pad=0),
    legend=dict(
        orientation="h", bgcolor='rgba(0, 0, 0, 0)',
        yanchor="top", y=-0.2, xanchor="center", x=0.5,
        font=AXIS_TICK_FONT, itemwidth=90,
    ),
    showlegend=False,
)

# save and show figure
sac_sensitivity_fig.write_image(os.path.join(OUT_PATH, "sac_sensitivity.png"), scale=3,)
sac_sensitivity_fig.show()

## Onset/Offset Comparison

In [43]:
with open(os.path.join(LUND_PATH, "fig6-fixation.json"), 'r') as f:
    lund_fig = pio.from_json(f.read())
with open(os.path.join(HFC_PATH, "fig6-fixation-hfc.json"), 'r') as f:
    hfc_fig = pio.from_json(f.read())

row_titles = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
subplot_titles = ["Ann.MN", "All Detectors", "ENGBERT", "REMoDNaV"] * 2
onset_offset_fig = make_subplots(
    rows=len(row_titles), cols=1,
    row_titles=row_titles,
    shared_xaxes=True, shared_yaxes=False,
    vertical_spacing=0.0,
)

# add traces
lund_subplot_map = {"Ann.MN": 1, "All Detectors": 2, "ENGBERT": 5, "REMoDNaV": 6}
for tr in lund_fig.data:
    if tr["yaxis"] != "y":  # ignore bottom-row MN-as-GT traces
        continue
    if tr["legendgroup"] not in ["2<sup>nd</sup> Ann.", "ENGBERT", "REMoDNaV", "All Detectors"]:    # ignore irrelevant detectors
        continue
    if tr["legendgroup"] == "2<sup>nd</sup> Ann.":
        tr["name"] = tr["legendgroup"] = "Ann. MN"
        tr["x"] = [["Ann. MN", "Ann. MN"], ["onset", "offset"]]
    if tr["legendgroup"] != "All Detectors":
        tr["opacity"] = max(tr["opacity"], 0.5)
    onset_offset_fig.add_trace(tr, row=1, col=1)
for tr in hfc_fig.data:
    if tr["yaxis"] != "y2":  # ignore bottom-row MN-as-GT traces
        continue
    if tr["legendgroup"] not in ["2<sup>nd</sup> Ann.", "ENGBERT", "REMoDNaV", "All Detectors"]:    # ignore irrelevant detectors
        continue
    if tr["legendgroup"] == "2<sup>nd</sup> Ann.":
        tr["name"] = tr["legendgroup"] = "Ann. MN"
        tr["x"] = [["Ann. MN", "Ann. MN"], ["onset", "offset"]]
    if tr["legendgroup"] != "All Detectors":
        tr["opacity"] = max(tr["opacity"], 0.5)
    onset_offset_fig.add_trace(tr, row=2, col=1)

# update layout
onset_offset_fig.for_each_annotation(lambda ann: ann.update(
    font=SUBTITLE_FONT, textangle=0,
    x=0.0 , xanchor="left",
    y=1.0 if ann.text==row_titles[0] else 0.5, yanchor="middle",
))
onset_offset_fig.for_each_yaxis(lambda yax: yax.update(
    title=dict(text="<i>d'</i>", font=AXIS_LABEL_FONT, standoff=4*AXIS_LABEL_STANDOFF),
    showline=False,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    range=[-1.499, 5.9],
    tickfont=AXIS_TICK_FONT, tickmode="linear", tick0=0, dtick=1.5,
))
onset_offset_fig.for_each_xaxis(lambda xax: xax.update(
    title=dict(font=AXIS_LABEL_FONT),
    showline=False,
    showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT,
    ticks=None, dividercolor="rgba(0, 0, 0, 0)",
))
onset_offset_fig.update_layout(
    width=1000, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, b=0, t=7.5, pad=0),
    legend=dict(
        orientation="h", bgcolor='rgba(0, 0, 0, 0)',
        yanchor="top", y=-0.2, xanchor="center", x=0.5,
        font=AXIS_TICK_FONT, itemwidth=90,
    ),
    showlegend=False,
)

# save and show figure
onset_offset_fig.write_image(os.path.join(OUT_PATH, "onset_offset.png"), scale=3,)
onset_offset_fig.show()